# 1 Load Data

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_csv('C:/Users/zhou/Desktop/BIA 660/000--Project/3 - final/ted_talks_data_clean_train.csv')

df_train['topic+desc'] = df_train[['topics','descriptions']].agg(' '.join, axis=1)
df_train.head(3)
df_train['topic+desc'][0]

,speakers,durations,topics,views,descriptions,tags,topic+desc
0,James Veitch,0:09:48,This is what happens when you reply to spam email,63440723,"Suspicious emails: unclaimed insurance bonds, ...",comedy|curiosity|communication|humor|technology,This is what happens when you reply to spam em...
1,Amy Cuddy,0:21:02,Your body language may shape who you are,59281283,(NOTE: Some of the findings presented in this ...,body language|brain|business|psychology|self|s...,Your body language may shape who you are (NOTE...
2,Simon Sinek,0:18:04,How great leaders inspire action,52322192,Simon Sinek has a simple but powerful model fo...,TEDx|business|entrepreneur|leadership|success,How great leaders inspire action Simon Sinek h...


'This is what happens when you reply to spam email Suspicious emails: unclaimed insurance bonds, diamond-encrusted safe deposit boxes, close friends marooned in a foreign country. They pop up in our inboxes, and standard procedure is to delete on sight. But what happens when you reply? Follow along as writer and comedian James Veitch narrates a hilarious, weeks-long exchange with a spammer who offered to cut him in on a hot deal.'

In [22]:
df_test = pd.read_csv('C:/Users/zhou/Desktop/BIA 660/000--Project/3 - final/ted_talks_data_clean_test.csv')

df_test['topic+desc'] = df_test['topics'].astype(str) +df_test['descriptions'].astype(str)
df_test.head(3)
df_test['topic+desc'][0]

,speakers,durations,views,topics,descriptions,tags,topic+desc
0,Kevin Slavin,0:15:22,4238935,How algorithms shape our world,"We live in a world run by algorithms, computer...",technology,How algorithms shape our worldWe live in a wor...
1,Dan Dennett,0:21:48,4232675,The illusion of consciousness,Philosopher Dan Dennett makes a compelling arg...,psychology,The illusion of consciousnessPhilosopher Dan D...
2,Erik Johansson,0:06:21,4227715,Impossible photography,Erik Johansson creates realistic photos of imp...,technology,Impossible photographyErik Johansson creates r...


'How algorithms shape our worldWe live in a world run by algorithms, computer programs that make decisions or solve problems for us. In this riveting, funny talk, Kevin Slavin shows how modern algorithms determine stock prices, espionage tactics, even the movies you watch. But, he asks: If we depend on complex algorithms to manage our daily decisions -- when do we start to lose control?'

# 2 Kmeans - Cosine Distance

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

tfidf_vect = TfidfVectorizer(stop_words="english", min_df=5) 
dtm= tfidf_vect.fit_transform(df_train['topic+desc'])
print (dtm.shape)

(4164, 4853)


In [25]:
from nltk.cluster import KMeansClusterer, cosine_distance

num_clusters = 14

model_1 = KMeansClusterer(num_clusters, cosine_distance, repeats=20)
clusters_1 = model_1.cluster(dtm.toarray(), assign_clusters=True)

In [26]:
print(clusters_1[:10])

[9, 3, 7, 3, 10, 8, 5, 5, 7, 5]


In [27]:
from sklearn import metrics

centroids=np.array(model_1.means())
sorted_centroids = centroids.argsort(axis=-1)[:, ::-1]

voc_lookup= tfidf_vect.get_feature_names()

for i in range(num_clusters):
    top_words=[voc_lookup[word_index]  for word_index in sorted_centroids[i, :20]]
    print("Cluster %d:\n%s " % (i, ", ".join(top_words)))

Cluster 0:
ted, music, performance, musical, dance, poem, art, ed, sound, fellow, animation, play, new, song, stage, songs, video, plays, singer, artist 
Cluster 1:
design, designer, david, architect, art, nature, designs, life, products, new, designers, shows, rockwell, talk, inspired, just, buildings, build, great, architecture 
Cluster 2:
world, author, talk, life, eye, game, says, shows, shares, make, real, people, talks, opening, film, ted, hidden, way, photos, new 
Cluster 3:
brain, love, talk, shares, new, funny, language, says, way, life, neuroscientist, personal, stories, understand, brains, help, compassion, research, experience, human 
Cluster 4:
story, women, tells, talk, kids, life, school, stories, men, young, war, child, powerful, world, shares, says, girls, personal, violence, prison 
Cluster 5:
time, work, know, says, sleep, talk, shares, life, good, need, space, happiness, don, makes, make, best, career, way, ways, things 
Cluster 6:
like, human, new, shows, robots, a

In [28]:
test_dtm = tfidf_vect.transform(df_test['topic+desc'])

predicted_1 = [model_1.classify(v) for v in test_dtm.toarray()]

predicted_1[0:10]

[6, 5, 2, 6, 0, 3, 2, 0, 12, 5]

In [29]:
label_vs_clusterID_1 = pd.DataFrame(list(zip(df_test["tags"].values, predicted_1)),\
                            columns = ["label", "cluster"])
label_vs_clusterID_1.head()

pd.crosstab( index=label_vs_clusterID_1.cluster, columns=label_vs_clusterID_1.label)

,label,cluster
0,technology,6
1,psychology,5
2,technology,2
3,science,6
4,art,0


label,art,biology,business,culture,economics,education,health,medical research,personal growth,psychology,religion,science,society,technology
cluster,,,,,,,,,,,,,,
0,5,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,0,0,1,1,1,1,0,1,1,1,1,1,2
3,1,2,0,3,0,1,1,1,2,7,0,1,0,0
4,0,0,1,2,0,1,0,0,1,0,0,0,1,0
5,1,1,4,2,1,0,3,0,4,6,1,2,3,1
6,0,1,1,1,0,0,1,0,0,1,0,1,0,6
7,0,1,0,0,1,0,0,0,0,1,0,2,0,0
8,0,2,0,0,0,1,0,0,0,0,0,5,0,0


# 3 Kmeans - Euclidean Distance

In [30]:
from sklearn.cluster import KMeans

model_2 = KMeans(n_clusters=num_clusters, n_init=20, random_state = 42).fit(dtm)
clusters_2 = model_2.labels_.tolist()

In [31]:
print(clusters_2[:10])

[4, 5, 4, 5, 5, 5, 7, 9, 5, 9]


In [ ]:
predicted_2 = model_2.predict(test_dtm)

In [40]:
# metrics.silhouette_score(df_train['topic+desc'], predicted_2, metric='euclidean')

In [33]:


label_vs_clusterID_2 = pd.DataFrame(list(zip(df_test["tags"].values, predicted_2)),\
                            columns = ["label", "cluster"])
label_vs_clusterID_2.head()

pd.crosstab( index=label_vs_clusterID_2.cluster, columns=label_vs_clusterID_2.label)

,label,cluster
0,technology,13
1,psychology,5
2,technology,9
3,science,13
4,art,4


label,art,biology,business,culture,economics,education,health,medical research,personal growth,psychology,religion,science,society,technology
cluster,,,,,,,,,,,,,,
0,3,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,1,3,0,5,0,2,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,2,0,0,0,0,0,0,0
4,6,0,0,2,0,1,0,0,0,0,0,0,0,0
5,1,1,3,7,2,1,3,0,7,8,1,1,10,1
7,0,0,0,0,0,0,1,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,2,1,1
9,0,2,2,0,0,1,1,0,1,3,1,2,0,2


# 4 GMM

In [34]:
from sklearn import mixture

lowest_bic = np.infty   
model_3 = None

cv_types = ['spherical', 'tied', 'diag'] 

for cv_type in cv_types:

    gmm = mixture.GaussianMixture(n_components=num_clusters, covariance_type=cv_type, random_state=42)
    gmm.fit(dtm.toarray())
        
    bic = gmm.bic(dtm.toarray()) 
        
    if bic < lowest_bic: 
        lowest_bic = bic
        model_3 = gmm


GaussianMixture(covariance_type='spherical', n_components=14, random_state=42)

GaussianMixture(covariance_type='tied', n_components=14, random_state=42)

GaussianMixture(covariance_type='diag', n_components=14, random_state=42)

In [35]:
model_3

GaussianMixture(covariance_type='diag', n_components=14, random_state=42)

In [36]:
predicted_3 = model_3.predict(test_dtm.toarray())
predicted[0:10]

[7, 1, 6, 7, 8, 0, 6, 12, 7, 6]

In [37]:
label_vs_clusterID_3 = pd.DataFrame(list(zip(df_test["tags"].values, predicted_3)), columns = ["label", "cluster"])
label_vs_clusterID_3.head()

pd.crosstab( index=label_vs_clusterID_3.cluster, columns=label_vs_clusterID_3.label)

,label,cluster
0,technology,10
1,psychology,4
2,technology,0
3,science,5
4,art,0


label,art,biology,business,culture,economics,education,health,medical research,personal growth,psychology,religion,science,society,technology
cluster,,,,,,,,,,,,,,
0,9,3,2,4,1,2,1,4,1,2,0,1,1,6
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,2,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,1,1,0,1,0,1,0,0
4,1,3,0,3,1,2,2,0,1,2,0,2,0,1
5,0,0,0,0,0,0,0,0,0,1,0,1,0,1
10,0,1,5,2,1,0,1,1,4,11,0,8,3,3
11,1,1,1,3,0,0,3,0,0,0,2,1,6,3
13,0,0,0,0,0,0,1,0,3,0,0,0,2,0
